In [1]:
import pandas as pd
from data import preprocessing
import pandas as pd
import numpy as np
import time
from stable_baselines.common.vec_env import DummyVecEnv

from model.models import *

%load_ext autoreload

%autoreload 2

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
df = preprocessing.load_dataset(file_name= '../cryptocoin_all_ohlc.csv')

In [3]:
df = df.iloc[:,1:]

In [4]:
df.Date = pd.to_datetime(df.Date)

In [5]:
unique_trade_date = df[(df.Date > '2017-01-01')&(df.Date <= '2020-10-10')].Date.unique()

In [6]:
df[df.Date=='2017-01-01'].ticker

8331                 Ardor
18036              bitcoin
19432            BitShares
31261               Decred
32748             DigiByte
35780             Dogecoin
43707     Ethereum Classic
45103             Ethereum
49544                Golem
70627                 Lisk
72023             Litecoin
74310         MaidSafeCoin
79902             MonaCoin
81298               Monero
84701                  NEM
86097                  Neo
101497                Redd
106554             Siacoin
110617             Stellar
113227             Stratis
118010             Syscoin
120210              Tether
133825               Verge
137329               Waves
142043                 XRP
144301               Zcash
145697               Zcoin
Name: ticker, dtype: object

In [26]:
stocks=['bitcoin','Ethereum','Ripple','NEM','Litecoin','Waves','Stellar','DigiByte','Monero','Neo','Stratis','Golem','Zcash','OMG','Dogecoin']
df_cr_sub  = df[df["ticker"].str.contains('|'.join(stocks))]
df_cr_sub.head()

,Close,Date,High,Low,Open,Volume,ticker
16641,"13,654.22",2020-10-27,"13,759.67","13,060.84","13,075.24","33,749,878,156",bitcoin
16642,"13,075.25",2020-10-26,"13,225.30","12,822.38","13,031.20","29,461,458,313",bitcoin
16643,"13,031.17",2020-10-25,"13,329.18","12,910.06","13,108.06","24,406,920,575",bitcoin
16644,"13,108.06",2020-10-24,"13,145.07","12,885.75","12,931.57","24,542,317,940",bitcoin
16645,"12,931.54",2020-10-23,"13,015.96","12,752.65","12,971.55","28,974,975,003",bitcoin


In [27]:
# Turn close price from string to float
df_cr_sub['Adj Close'] = df_cr_sub['Close']
for col in ['Close','High','Low','Open','Volume','Adj Close']:
    df_cr_sub[col] = df_cr_sub[col].apply(lambda x: round(float(x.replace(',','')),5))

C:\Users\kaanb\Anaconda3\envs\nlpfast\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\kaanb\Anaconda3\envs\nlpfast\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [28]:
df_cr_sub.head()

,Close,Date,High,Low,Open,Volume,ticker,Adj Close
16641,13654.22,2020-10-27,13759.67,13060.84,13075.24,3.374988e+10,bitcoin,13654.22
16642,13075.25,2020-10-26,13225.30,12822.38,13031.20,2.946146e+10,bitcoin,13075.25
16643,13031.17,2020-10-25,13329.18,12910.06,13108.06,2.440692e+10,bitcoin,13031.17
16644,13108.06,2020-10-24,13145.07,12885.75,12931.57,2.454232e+10,bitcoin,13108.06
16645,12931.54,2020-10-23,13015.96,12752.65,12971.55,2.897498e+10,bitcoin,12931.54


In [29]:
df_cr_sub.fillna(0,inplace=True)

C:\Users\kaanb\Anaconda3\envs\nlpfast\lib\site-packages\pandas\core\frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [30]:
len(df_cr_sub.ticker.unique())

15

In [31]:
df_cr_sub = preprocessing.add_technical_indicator(df_cr_sub)

NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values
C:\Users\kaanb\Anaconda3\envs\nlpfast\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\kaanb\Anaconda3\envs\nlpfast\lib\site-packages\stockstats.py:474: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  lambda x: np.fabs(x - x.mean()).mean())
NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values
NOTE: Behavior of MACDH calculatio

In [32]:
# Add turbulance indicator
df_cr_sub = preprocessing.add_turbulance(df_cr_sub)

C:\Users\kaanb\Desktop\Code\Finance\Bot\RLBot\data\preprocessing.py:148: RuntimeWarning: invalid value encountered in greater
  if temp > 0:


In [33]:
# rebalance_window is the number of months to retrain the model
# validation_window is the numebr of months to validation the model and select for trading
rebalance_window = 63
validation_window = 63

In [34]:
df_cr_sub.columns = ['Low','Date','High','Open','Close','volume','ticker','adjcp','macd','rsi','cci','adx','turbulence']
df_cr_sub = df_cr_sub.fillna(0)
df_cr_sub.head()


,Low,Date,High,Open,Close,volume,ticker,adjcp,macd,rsi,cci,adx,turbulence
0,0.00026,2017-01-01,0.00027,0.00024,0.00025,17929.2,DigiByte,0.00026,0.0,0.0,0.0,0.0,0.0
1,0.00023,2017-01-01,0.00023,0.00022,0.00022,136829.0,Dogecoin,0.00023,0.0,0.0,0.0,0.0,0.0
2,8.17000,2017-01-01,8.47000,7.98000,7.98000,14731700.0,Ethereum,8.17000,0.0,0.0,0.0,0.0,0.0
3,1.40000,2017-01-01,1.50000,1.35000,1.41000,3570370.0,Ethereum Classic,1.40000,0.0,0.0,0.0,0.0,0.0
4,0.00987,2017-01-01,0.00996,0.00917,0.00973,22571.1,Golem,0.00987,0.0,0.0,0.0,0.0,0.0


In [47]:
df_cr_sub = pd.read_csv('../preprocessed_crypto.csv')

In [48]:
df_cr_sub.ticker.unique()

array(['DigiByte', 'Dogecoin', 'Ethereum', 'Ethereum Classic', 'Golem',
       'Litecoin', 'Monero', 'NEM', 'Neo', 'Stellar', 'Stratis', 'Waves',
       'Zcash', 'bitcoin', 'OMG Network'], dtype=object)

In [37]:
#df_cr_sub.rename(columns={'Volume':'volume'}, inplace=True)
df_cr_sub.head()

,Low,Date,High,Open,Close,volume,ticker,adjcp,macd,rsi,cci,adx,turbulence
0,0.00026,2017-01-01,0.00027,0.00024,0.00025,17929.2,DigiByte,0.00026,0.0,0.0,0.0,0.0,0.0
1,0.00023,2017-01-01,0.00023,0.00022,0.00022,136829.0,Dogecoin,0.00023,0.0,0.0,0.0,0.0,0.0
2,8.17000,2017-01-01,8.47000,7.98000,7.98000,14731700.0,Ethereum,8.17000,0.0,0.0,0.0,0.0,0.0
3,1.40000,2017-01-01,1.50000,1.35000,1.41000,3570370.0,Ethereum Classic,1.40000,0.0,0.0,0.0,0.0,0.0
4,0.00987,2017-01-01,0.00996,0.00917,0.00973,22571.1,Golem,0.00987,0.0,0.0,0.0,0.0,0.0


In [38]:
df_cr_sub.ticker.unique()

array(['DigiByte', 'Dogecoin', 'Ethereum', 'Ethereum Classic', 'Golem',
       'Litecoin', 'Monero', 'NEM', 'Neo', 'Stellar', 'Stratis', 'Waves',
       'Zcash', 'bitcoin', 'OMG Network'], dtype=object)

In [50]:
len(df_cr_sub.ticker.unique())

15

In [55]:
len(df_cr_sub[(df_cr_sub.Date == '2017-10-02')].ticker)

15

In [63]:
for d in df_cr_sub.Date:
    
    if len(df_cr_sub[(df_cr_sub.Date == d)].ticker) != 15:
        print(d)

2017-01-01
2017-01-01
2017-01-01
2017-01-01
2017-01-01
2017-01-01
2017-01-01
2017-01-01
2017-01-01
2017-01-01
2017-01-01
2017-01-01
2017-01-01
2017-01-01
2017-01-02
2017-01-02
2017-01-02
2017-01-02
2017-01-02
2017-01-02
2017-01-02
2017-01-02
2017-01-02
2017-01-02
2017-01-02
2017-01-02
2017-01-02
2017-01-02
2017-01-03
2017-01-03
2017-01-03
2017-01-03
2017-01-03
2017-01-03
2017-01-03
2017-01-03
2017-01-03
2017-01-03
2017-01-03
2017-01-03
2017-01-03
2017-01-03
2017-01-04
2017-01-04
2017-01-04
2017-01-04
2017-01-04
2017-01-04
2017-01-04
2017-01-04
2017-01-04
2017-01-04
2017-01-04
2017-01-04
2017-01-04
2017-01-04
2017-01-05
2017-01-05
2017-01-05
2017-01-05
2017-01-05
2017-01-05
2017-01-05
2017-01-05
2017-01-05
2017-01-05
2017-01-05
2017-01-05
2017-01-05
2017-01-05
2017-01-06
2017-01-06
2017-01-06
2017-01-06
2017-01-06
2017-01-06
2017-01-06
2017-01-06
2017-01-06
2017-01-06
2017-01-06
2017-01-06
2017-01-06
2017-01-06
2017-01-07
2017-01-07
2017-01-07
2017-01-07
2017-01-07
2017-01-07
2017-01-07

2017-02-27
2017-02-28
2017-02-28
2017-02-28
2017-02-28
2017-02-28
2017-02-28
2017-02-28
2017-02-28
2017-02-28
2017-02-28
2017-02-28
2017-02-28
2017-02-28
2017-02-28
2017-03-01
2017-03-01
2017-03-01
2017-03-01
2017-03-01
2017-03-01
2017-03-01
2017-03-01
2017-03-01
2017-03-01
2017-03-01
2017-03-01
2017-03-01
2017-03-01
2017-03-02
2017-03-02
2017-03-02
2017-03-02
2017-03-02
2017-03-02
2017-03-02
2017-03-02
2017-03-02
2017-03-02
2017-03-02
2017-03-02
2017-03-02
2017-03-02
2017-03-03
2017-03-03
2017-03-03
2017-03-03
2017-03-03
2017-03-03
2017-03-03
2017-03-03
2017-03-03
2017-03-03
2017-03-03
2017-03-03
2017-03-03
2017-03-03
2017-03-04
2017-03-04
2017-03-04
2017-03-04
2017-03-04
2017-03-04
2017-03-04
2017-03-04
2017-03-04
2017-03-04
2017-03-04
2017-03-04
2017-03-04
2017-03-04
2017-03-05
2017-03-05
2017-03-05
2017-03-05
2017-03-05
2017-03-05
2017-03-05
2017-03-05
2017-03-05
2017-03-05
2017-03-05
2017-03-05
2017-03-05
2017-03-05
2017-03-06
2017-03-06
2017-03-06
2017-03-06
2017-03-06
2017-03-06

2017-04-30
2017-04-30
2017-04-30
2017-04-30
2017-04-30
2017-04-30
2017-04-30
2017-04-30
2017-04-30
2017-04-30
2017-04-30
2017-04-30
2017-05-01
2017-05-01
2017-05-01
2017-05-01
2017-05-01
2017-05-01
2017-05-01
2017-05-01
2017-05-01
2017-05-01
2017-05-01
2017-05-01
2017-05-01
2017-05-01
2017-05-02
2017-05-02
2017-05-02
2017-05-02
2017-05-02
2017-05-02
2017-05-02
2017-05-02
2017-05-02
2017-05-02
2017-05-02
2017-05-02
2017-05-02
2017-05-02
2017-05-03
2017-05-03
2017-05-03
2017-05-03
2017-05-03
2017-05-03
2017-05-03
2017-05-03
2017-05-03
2017-05-03
2017-05-03
2017-05-03
2017-05-03
2017-05-03
2017-05-04
2017-05-04
2017-05-04
2017-05-04
2017-05-04
2017-05-04
2017-05-04
2017-05-04
2017-05-04
2017-05-04
2017-05-04
2017-05-04
2017-05-04
2017-05-04
2017-05-05
2017-05-05
2017-05-05
2017-05-05
2017-05-05
2017-05-05
2017-05-05
2017-05-05
2017-05-05
2017-05-05
2017-05-05
2017-05-05
2017-05-05
2017-05-05
2017-05-06
2017-05-06
2017-05-06
2017-05-06
2017-05-06
2017-05-06
2017-05-06
2017-05-06
2017-05-06

2017-06-23
2017-06-23
2017-06-23
2017-06-23
2017-06-23
2017-06-23
2017-06-23
2017-06-23
2017-06-23
2017-06-24
2017-06-24
2017-06-24
2017-06-24
2017-06-24
2017-06-24
2017-06-24
2017-06-24
2017-06-24
2017-06-24
2017-06-24
2017-06-24
2017-06-24
2017-06-24
2017-06-25
2017-06-25
2017-06-25
2017-06-25
2017-06-25
2017-06-25
2017-06-25
2017-06-25
2017-06-25
2017-06-25
2017-06-25
2017-06-25
2017-06-25
2017-06-25
2017-06-26
2017-06-26
2017-06-26
2017-06-26
2017-06-26
2017-06-26
2017-06-26
2017-06-26
2017-06-26
2017-06-26
2017-06-26
2017-06-26
2017-06-26
2017-06-26
2017-06-27
2017-06-27
2017-06-27
2017-06-27
2017-06-27
2017-06-27
2017-06-27
2017-06-27
2017-06-27
2017-06-27
2017-06-27
2017-06-27
2017-06-27
2017-06-27
2017-06-28
2017-06-28
2017-06-28
2017-06-28
2017-06-28
2017-06-28
2017-06-28
2017-06-28
2017-06-28
2017-06-28
2017-06-28
2017-06-28
2017-06-28
2017-06-28
2017-06-29
2017-06-29
2017-06-29
2017-06-29
2017-06-29
2017-06-29
2017-06-29
2017-06-29
2017-06-29
2017-06-29
2017-06-29
2017-06-29

In [78]:
unique_trade_date = df_cr_sub[(df_cr_sub.Date > '2018-04-02')&(df_cr_sub.Date <= '2020-10-30')].Date.unique()


In [82]:
start_date = unique_trade_date[0]
end_date = '2020-10-27'

In [83]:
unique_trade_date

array(['2018-04-03', '2018-04-04', '2018-04-05', '2018-04-06',
       '2018-04-07', '2018-04-08', '2018-04-09', '2018-04-10',
       '2018-04-11', '2018-04-12', '2018-04-13', '2018-04-14',
       '2018-04-15', '2018-04-16', '2018-04-17', '2018-04-18',
       '2018-04-19', '2018-04-20', '2018-04-21', '2018-04-22',
       '2018-04-23', '2018-04-24', '2018-04-25', '2018-04-26',
       '2018-04-27', '2018-04-28', '2018-04-29', '2018-04-30',
       '2018-05-01', '2018-05-02', '2018-05-03', '2018-05-04',
       '2018-05-05', '2018-05-06', '2018-05-07', '2018-05-08',
       '2018-05-09', '2018-05-10', '2018-05-11', '2018-05-12',
       '2018-05-13', '2018-05-14', '2018-05-15', '2018-05-16',
       '2018-05-17', '2018-05-18', '2018-05-19', '2018-05-20',
       '2018-05-21', '2018-05-22', '2018-05-23', '2018-05-24',
       '2018-05-25', '2018-05-26', '2018-05-27', '2018-05-28',
       '2018-05-29', '2018-05-30', '2018-05-31', '2018-06-01',
       '2018-06-02', '2018-06-03', '2018-06-04', '2018-

In [84]:
run_ensemble_strategy(df=df_cr_sub, 
                          unique_trade_date= unique_trade_date,
                          rebalance_window = rebalance_window,
                          validation_window=validation_window,start_date= start_date,
                          end_date=end_date,
                          load=True)

KeyError: 0

In [196]:
start_test = '2016-01-10'
end_test = '2020-09-30'